In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import pytraj as pt
import nglview as nv

import matplotlib
from matplotlib import pyplot as plt

import cphmdanalysis as cphmda

In [80]:
nRuns=4
sysName='WT'
with open('pH_List.txt','r') as pHlistFile:
    rep_pH_vals=list(map(float,pHlistFile.readline().split(', ')))
phList=rep_pH_vals*nRuns
nReps=len(rep_pH_vals)
lambdaFileNames=['%s.rep%02g.%02g.lambda'%(sysName,irep+1,irun+1) \
                 for irun in np.arange(nRuns) \
                 for irep in np.arange(len(rep_pH_vals))]
lambdaFiles=[cphmda.lambda_file(filename) for filename in lambdaFileNames]

for lambda_file in lambdaFiles:
    lambda_file.compute_s_values()

pka_data=cphmda.compute_pkas(phList,lambdaFiles) #wrapped_lambda_files)

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
orig_struc=pt.load("../../../structures/WT.pdb")
pKaTable=pd.DataFrame([(tempStruc.top.residue(res-1).name,
 (tempStruc.top.residue(res-1)).original_resid,
 (orig_struc.top.residue(res-1)).original_resid+1,
  "%.3f"%(
      pka_data[ires][2]),
  "%.3f"%(
      pka_data[ires][0])) for ires,res in enumerate(lambdaFiles[0].titrreses)],
            columns=['ResName','Resid','PDB_Resid','pKa','Hill_Coef'])
pKaTable.to_csv('pKa_DataTable.csv',index=False)
goodRows=np.sum((pKaTable.pKa!='nan'))
totalRows=pKaTable.shape[0]
print('Total Number of Residues: ',
      totalRows,
      '\nNumber of succesfully titrated residues: ',
      goodRows,
      '\nNumber of unsuccesfully titrated residues: ',
      totalRows-goodRows)
pKaTable.head()

Total Number of Residues:  95 
Number of succesfully titrated residues:  70 
Number of unsuccesfully titrated residues:  25


,ResName,Resid,PDB_Resid,pKa,Hill_Coef
0,AS2,6,178,2.946,0.718
1,AS2,11,183,3.284,0.583
2,AS2,14,186,3.322,0.766
3,HIP,15,187,6.872,0.860
4,CYS,17,189,9.674,0.496


In [78]:
for resType in pKaTable.ResName.unique():
    print(pKaTable[pKaTable.ResName == resType].sort_values("pKa"))

   ResName  Resid  PDB_Resid    pKa Hill_Coef
31     AS2    165        336  0.942     0.376
40     AS2    198        369  1.911     0.463
52     AS2    255        426  2.071     0.723
57     AS2    272        443  2.177     1.050
88     AS2    407         80  2.284     0.855
38     AS2    183        354  2.425     0.573
80     AS2    369         42  2.530     0.840
47     AS2    226        397  2.611     0.748
23     AS2    122        293  2.662     0.928
79     AS2    365         38  2.750     2.266
76     AS2    360         33  2.765     0.790
0      AS2      6        177  2.946     0.718
45     AS2    223        394  3.020     0.899
71     AS2    339         12  3.112     0.995
61     AS2    293        464  3.279     0.826
1      AS2     11        182  3.284     0.583
91     AS2    428        101  3.321     0.618
2      AS2     14        185  3.322     0.766
54     AS2    262        433  3.332     0.746
11     AS2     70        241  3.415     0.760
69     AS2    328        499  3.44